In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


### 加载数据

In [ ]:

# 请替换成你自己的CSV或Excel文件路径
# 这里以假设的DataFrame为例
# df = pd.read_csv('your_data.csv') 
# 注意如果是中文表头，记得加 encoding="utf-8-sig" 或 gbk


## 1. 东中西东北板块社融走势比较

In [ ]:

region_groups = {
    "东部": ["北京", "天津", "河北", "上海", "江苏", "浙江", "广东", "福建", "山东", "海南"],
    "中部": ["山西", "安徽", "江西", "河南", "湖北", "湖南"],
    "西部": ["内蒙古", "广西", "重庆", "四川", "贵州", "云南", "西藏", "陕西", "甘肃", "青海", "宁夏", "新疆"],
    "东北": ["辽宁", "吉林", "黑龙江"]
}

group_data = {}
for group, provinces in region_groups.items():
    common_provinces = [p for p in provinces if p in df.columns]
    group_data[group] = df[common_provinces].sum(axis=1)

group_df = pd.DataFrame(group_data)

plt.figure(figsize=(12,8))
for group in group_df.columns:
    sns.lineplot(x=group_df.index, y=group_df[group], label=group, marker="o")
plt.title("东中西东北板块社融规模变化（2018-2024）", fontsize=18)
plt.xlabel("年份", fontsize=14)
plt.ylabel("社会融资规模总额（亿元）", fontsize=14)
plt.legend(title="板块")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 2. 2018-2024年累计增长率分析

In [ ]:

growth_rate = (df.loc["2024"] - df.loc["2018"]) / df.loc["2018"] * 100
growth_rate = growth_rate.sort_values(ascending=False)

plt.figure(figsize=(14,10))
sns.barplot(x=growth_rate.values, y=growth_rate.index, palette="coolwarm")
plt.title("2018-2024年社会融资规模累计增长率", fontsize=18)
plt.xlabel("增长率（%）", fontsize=14)
plt.ylabel("地区", fontsize=14)
plt.tight_layout()
plt.show()

print("增长率Top5:")
print(growth_rate.head())

print("\n增长率Bottom5:")
print(growth_rate.tail())


## 3. 2018-2024年波动性（标准差）分析

In [ ]:

volatility = df.std()
volatility = volatility.sort_values(ascending=False)

plt.figure(figsize=(14,10))
sns.barplot(x=volatility.values, y=volatility.index, palette="mako")
plt.title("2018-2024年社会融资规模波动性（标准差）", fontsize=18)
plt.xlabel("标准差", fontsize=14)
plt.ylabel("地区", fontsize=14)
plt.tight_layout()
plt.show()

print("波动性Top5:")
print(volatility.head())

print("\n波动性Bottom5:")
print(volatility.tail())


## 4. 基于社融特征的聚类分析

In [ ]:

X = df.T.values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=4, random_state=0)
clusters = kmeans.fit_predict(X_scaled)

cluster_result = pd.DataFrame({
    "地区": df.columns,
    "聚类类别": clusters
}).sort_values("聚类类别")

print(cluster_result)

plt.figure(figsize=(12,8))
sns.scatterplot(x=X_scaled[:,0], y=X_scaled[:,1], hue=clusters, palette="Set2", s=100)
for i, txt in enumerate(df.columns):
    plt.annotate(txt, (X_scaled[i,0]+0.02, X_scaled[i,1]+0.02), fontsize=9)
plt.title("各省社融特征聚类分布", fontsize=18)
plt.xlabel("特征1（标准化后）")
plt.ylabel("特征2（标准化后）")
plt.legend(title="类别")
plt.tight_layout()
plt.show()
